In [1]:
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

from selenium.webdriver.common.by import By
from selenium import webdriver

#### https://silkwaychina.ru/news/

request only

In [69]:
data = pd.DataFrame()

source = 'silkwaychina'

for year in list(range(2022, 2025)):

    try:
        # Каких-то страниц может не существовать
        for page_num in range(1, 5):
            
            base_url = f'https://silkwaychina.ru/news/?year={year}&PAGEN_1={page_num}'
        
            r = requests.get(base_url)
            
            soup_all = BeautifulSoup(r.text, 'lxml')
    
            items = soup_all.find_all('div', class_=['item col-md-3 col-sm-4 col-xs-6'])
            
            for item in items:

                date = item.find("span", class_="date font_xs").text
                
                title = item.find("div", class_="title font_md").text.strip()
                
                link = item.find("a")["href"]
            
                href = f'https://silkwaychina.ru/news{link}'
            
                r = requests.get(href)
                
                soup = BeautifulSoup(r.text, 'lxml')
                
                content = soup.find('div', class_=['content'])
                
                text = ''.join([i.text.replace('• ', '').replace('\xa0', '') for i in content.find_all("p")])
                
                date = soup.find('div', class_=['detail news fixed_wrapper']).find('span', class_='date').text
            
                sub_data = pd.DataFrame([[source, title, date, href, text]], columns=['source', 'title', 'date', 'href', 'text'])
            
                data = pd.concat([data, sub_data], axis=0)

                time.sleep(1)
                
    except Exception as e:
        print(e)

data.to_excel(f'{source}.xlsx')

'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'


In [71]:
print(data.shape)
display(data)

(121, 5)


,source,title,date,href,text
0,silkwaychina,Важная информация - в Китае сняли все каранти...,23 декабря 2022,https://silkwaychina.ru/news/news/novosti/vazh...,На данный момент в Китае сняли все карантинные...
0,silkwaychina,Важная информация - праздничные выходные! 2023,20 декабря 2022,https://silkwaychina.ru/news/news/novosti/vazh...,"Уважаемые клиенты, впереди нас ждёт череда пра..."
0,silkwaychina,Складские хранений - популярная услуга в Китае,19 декабря 2022,https://silkwaychina.ru/news/news/statya/sklad...,\r\n\t Китайская сторона заинтересована в рыно...
0,silkwaychina,Скоро Китайский Новый год 2023,13 декабря 2022,https://silkwaychina.ru/news/news/novosti/uspe...,"Друзья, хотим напомнить что скоро Китайский Но..."
0,silkwaychina,Перевозки продуктов питания из Китая в РФ,11 декабря 2022,https://silkwaychina.ru/news/news/statya/perev...,\r\n\t Продукты питанияв настоящий момент не т...
...,...,...,...,...,...
0,silkwaychina,10 трендовых товаров на лето из Китая 2024,28 января 2024,https://silkwaychina.ru/news/news/statya/10-tr...,Узнайте о самых модных и популярных товарах из...
0,silkwaychina,11 необычных фактов о Китае,21 января 2024,https://silkwaychina.ru/news/news/statya/11-ne...,11 необычных и даже странных фактов о Китае - ...
0,silkwaychina,Как организовать сопровождение грузоперевозок ...,16 января 2024,https://silkwaychina.ru/news/news/statya/kak-o...,Сегодня многие компании ориентируются на поста...
0,silkwaychina,"Все, что вам нужно знать о доставке из Китая",10 января 2024,https://silkwaychina.ru/news/news/statya/vse-c...,Доставка из Китая может показаться сложной зад...


#### https://prc.today/

telethon + request

In [ ]:
from telethon import TelegramClient
import pandas as pd
import asyncio
import re

In [83]:
def extract_data(messages, channel_name):
    
    channel_data = pd.DataFrame()

    for current_message in messages:

        group = '@' + channel_name

        sender_id = current_message.sender_id

        message_id = current_message.id

        datetime = current_message.date.strftime('%Y-%m-%d %H:%M:%S')

        text = current_message.text

        message_data = pd.DataFrame([[group, sender_id, message_id, datetime, text]], columns = ['group', 'sender_id', 'message_id', 'datetime', 'views', 'shares', 'text'])

        channel_data = pd.concat([channel_data, message_data], axis=0)


    channel_data = channel_data.reset_index().drop('index', axis=1)

    channel_data.to_excel(f'{channel_name}.xlsx')

async def main():
    
    api_id = 
    api_hash = ''
    phone_number = ''

    client = TelegramClient('main', api_id, api_hash, system_version="1.01")

    await client.start()

    if not client.is_connected():
        await client.connect()

    channel_names = ['PRC_TODAY']

    for channel_name in channel_names:

        channel = await client.get_entity(channel_name)

        messages = [msg async for msg in client.iter_messages(channel)]

        extract_data(messages, channel_name='prc_today_tg')

    await client.log_out()

In [ ]:
# Парсинг тг канала

await main()

In [1]:
data = pd.DataFrame()

source = 'prc_roday'

# Парсинг статей по их ссылкам из тг канала. Т.к. в нем не полные тексты, но присутствуют ссылки на них. Из исходного сайта не удается получить данные
tg = pd.read_excel('prc_today_tg.xlsx', index_col=0)

tg = tg[tg.text.notnull()]

for index, row in tg.iterrows():
    
    try:
        urls = re.findall(r'https?://[^\s]+', row['text'])
    
        # Получаем последний URL
        last_url = urls[-1] if urls else None
    
        r = requests.get(last_url)
        
        soup = BeautifulSoup(r.text, 'lxml')
        
        elements = soup.find_all('p')
        
        for i, v in enumerate(elements):
            if 'подготовлена Порталом' in v.text:
                break
    
        title = row['text'].split('\n')[2]
    
        date = row['datetime']
        
        text = ''.join([i.text for i in elements[:i]])
    
        sub_data = pd.DataFrame([[source, title, date, href, text]], columns=['source', 'title', 'date', 'href', 'text'])
    
        data = pd.concat([data, sub_data], axis=0)

        time.sleep(1)
        
    except Exception as e:
        print(e)
        
data.to_excel(f'{source}.xlsx')

In [170]:
print(data.shape)
display(data)

(3722, 5)


,source,title,date,href,text
0,prc_roday,Экономика Китая устойчива в первом квартале,2024-04-03 16:41:43,https://www.india-briefing.com/news/top-sector...,Экономика Китая приветствовала уверенный старт...
0,prc_roday,Добыча угля в Китае достигла исторического мак...,2024-04-03 14:40:27,https://www.india-briefing.com/news/top-sector...,Добыча угля в Китае: объемы производства и имп...
0,prc_roday,"Компания Huawei заявляет о росте выручки на 9,...",2024-04-03 12:39:37,https://www.india-briefing.com/news/top-sector...,"Китайская компания Huawei сообщила 29 марта, ч..."
0,prc_roday,Немецкая оптическая компания ZEISS открывает ц...,2024-04-03 10:39:21,https://www.india-briefing.com/news/top-sector...,"ZEISS, немецкая транснациональная компания, ра..."
0,prc_roday,Качественное развитие китайской экономики – бл...,2024-04-03 08:43:59,https://www.india-briefing.com/news/top-sector...,Развитие китайской экономики: Высококачественн...
...,...,...,...,...,...
0,prc_roday,,2021-03-18 07:05:28,https://www.india-briefing.com/news/top-sector...,"Компания Guangzhou Asensing Technology Co Ltd,..."
0,prc_roday,,2021-03-17 17:39:53,https://www.india-briefing.com/news/top-sector...,"Компания Guangzhou Asensing Technology Co Ltd,..."
0,prc_roday,,2021-03-17 17:37:38,https://www.india-briefing.com/news/top-sector...,"Компания Guangzhou Asensing Technology Co Ltd,..."
0,prc_roday,,2021-03-17 15:21:48,https://www.india-briefing.com/news/top-sector...,"Компания Guangzhou Asensing Technology Co Ltd,..."


#### https://russian.news.cn/china/economic.htm

webdriver + requests

In [240]:
data = pd.DataFrame()

source = 'russian.news.cn'

driver = webdriver.Chrome()

base_url = "https://russian.news.cn/china/economic.htm"

driver.get(base_url)

time.sleep(3)

#Листаем страницу до конца
#while driver.find_element(By.ID, 'moreBtn').text == 'Загружено':
# driver.find_element(By.ID, 'moreBtn').click()
# time.sleep(1)

i = 0
while i < 15:
    driver.find_element(By.ID, 'moreBtn').click()
    time.sleep(1)
    i+=1

items = driver.find_elements(By.CSS_SELECTOR, '.list-item') + driver.find_elements(By.CSS_SELECTOR, '.clearfix')

for item in items:
    
    try:

        title = item.find_element(By.TAG_NAME, 'a').text

        description = item.text

        href = item.find_element(By.TAG_NAME, 'a').get_attribute('href')

        r = requests.get(href)
        
        soup = BeautifulSoup(r.text, 'lxml')
        
        date = soup.find('div', class_='info').text.strip().split('丨')[0]
        
        text = ' '.join([p.text for p in soup.find_all('p')][:-1])
    
        sub_data = pd.DataFrame([[source, title, date, href, description, text]], columns=['source', 'title', 'date', 'href', 'description', 'text'])
    
        data = pd.concat([data, sub_data], axis=0)

    except Exception as e:
        print(e)

# Закрыть браузер после выполнения
driver.quit()

# Сохранение файла
data.to_excel(f'{source}.xlsx')

'NoneType' object has no attribute 'text'
Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: chrome=123.0.6312.107); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF772337032+63090]
	(No symbol) [0x00007FF7722A2C82]
	(No symbol) [0x00007FF77213EC65]
	(No symbol) [0x00007FF77218499D]
	(No symbol) [0x00007FF772184ADC]
	(No symbol) [0x00007FF77217A0AC]
	(No symbol) [0x00007FF7721A701F]
	(No symbol) [0x00007FF77217A00A]
	(No symbol) [0x00007FF7721A71F0]
	(No symbol) [0x00007FF7721C3412]
	(No symbol) [0x00007FF7721A6D83]
	(No symbol) [0x00007FF7721783A8]
	(No symbol) [0x00007FF772179441]
	GetHandleVerifier [0x00007FF7727325AD+4238317]
	GetHandleVerifier [0x00007FF77276F70D+4488525]
	GetHandleVerifier [0x00007FF7727679EF+4456495]
	GetHandleVerifier [0x00007FF772410576+953270]
	(No symbol) [0x00007FF772

In [241]:
print(data.shape)
display(data)

(166, 6)


,source,title,date,href,description,text
0,russian.news.cn,Китай начал серийное производство изотопов угл...,2024-04-21 16:38:15,https://russian.news.cn/20240421/278723f0f5a04...,Китай начал серийное производство изотопов угл...,"Пекин, 21 апреля /Синьхуа/ -- Китай запустил с..."
0,russian.news.cn,Продажи коммерческих транспортных средств в Ки...,2024-04-21 15:53:15,https://russian.news.cn/20240421/bc945a7980c34...,Продажи коммерческих транспортных средств в Ки...,"Пекин, 21 апреля /Синьхуа/ -- В первом квартал..."
0,russian.news.cn,Объем добычи природного газа в Китае вырос в я...,2024-04-21 15:39:15,https://russian.news.cn/20240421/afa6fb2000224...,Объем добычи природного газа в Китае вырос в я...,"Пекин, 21 апреля /Синьхуа/ -- В первом квартал..."
0,russian.news.cn,Производство и продажи пикапов в Китае выросли...,2024-04-21 14:59:00,https://russian.news.cn/20240421/aa1cb965ab1f4...,Производство и продажи пикапов в Китае выросли...,"Пекин, 21 апреля /Синьхуа/ -- В первом квартал..."
0,russian.news.cn,Выработка электроэнергии в Китае в марте вырос...,2024-04-21 13:25:00,https://russian.news.cn/20240421/53cffb16560c4...,Выработка электроэнергии в Китае в марте вырос...,"Пекин, 21 апреля /Синьхуа/ -- В марте 2024 год..."
...,...,...,...,...,...,...
0,russian.news.cn,,2024-04-11 16:07:31,https://russian.news.cn/20240411/32b1395e01104...,,"Пекин, 11 апреля /Синьхуа/ -- Индекс цен произ..."
0,russian.news.cn,,2024-04-11 16:09:30,https://russian.news.cn/20240411/f43c7576cb4d4...,,"22 августа 2023 года, Пекин. Беспилотные дост..."
0,russian.news.cn,,2024-04-11 15:45:15,https://russian.news.cn/20240411/78be26d0df8d4...,,"Пекин, 11 апреля /Синьхуа/ -- Торги на фондовы..."
0,russian.news.cn,,2024-04-11 13:32:52,https://russian.news.cn/20240411/b3eebeac976a4...,,"Пекин, 11 апреля /Синьхуа/ -- Индекс потребите..."


### businessemirates

webdriver + requests

In [62]:
data = pd.DataFrame()

source = 'businessemirates'

driver = webdriver.Chrome()

for page in list(range(1, 40)):

    base_url = f'https://businessemirates.ae/news/?page={page}'
    
    driver.get(base_url)
    
    items = driver.find_elements(By.CSS_SELECTOR, '.ib_head')
    
    for item in items:
        
        title = item.find_element(By.TAG_NAME, 'a').text
        
        href  = item.find_element(By.TAG_NAME, 'a').get_attribute('href')

        if title!='':
            
            r = requests.get(href)
            soup = BeautifulSoup(r.text, 'lxml')
            
            title = soup.find('div', class_=['full_news_head']).text
            
            description = soup.find('div', class_=['full_news_desc']).text
            
            text = soup.find('div', class_=['full_news_text']).text
            
            date = soup.find('div', class_=['full_news_info']).text.split('\n')[4]
        
            sub_data = pd.DataFrame([[source, title, date, href, description, text]], columns=['source', 'title', 'date', 'href', 'description', 'text'])
    
            data = pd.concat([data, sub_data], axis=0)

# Закрыть браузер после выполнения
driver.quit()

# Сохранение файла
data.to_excel(f'{source}.xlsx')

In [68]:
data.shape

(819, 6)

### indianstartupnews

requests only

In [65]:
data = pd.DataFrame()

source = 'indianstartupnews'

for page in list(range(1, 30)):

        base_url = f'https://indianstartupnews.com/news?page={page}'
    
        r = requests.get(base_url)
    
        soup_all = BeautifulSoup(r.text, 'lxml')

        items = soup_all.find_all('div', class_=['col-sm-4 col-md-4 col-lg-4'])
        
        for item in items:
            
            try:
                # Находим элемент ссылки (a), содержащий нужную информацию
                link_tag = item.find('a', attrs={'aria-label': True})
                
                # Извлекаем URL из атрибута href
                href = link_tag['href'] if link_tag else None
                
                # Извлекаем текст заголовка
                title = link_tag.find('div', class_='post-title').get_text(strip=True) if link_tag else None
        
                # Находим тег <time> с классом "date"
                time_tag = soup.find('time', class_='date')
                
                # Извлекаем текст из найденного тега
                date = time_tag.get_text(strip=True) if time_tag else None
        
                url = 'https://indianstartupnews.com' + href
        
                r = requests.get(url)
                
                soup = BeautifulSoup(r.text, 'lxml')
                
                full_text = soup.find('div', class_=['article'])
                
                text = ' '.join([p.text for p in full_text.find_all('p')][:-1])
        
                sub_data = pd.DataFrame([[source, title, date, href, text]], columns=['source', 'title', 'date', 'href', 'text'])
            
                data = pd.concat([data, sub_data], axis=0)

            except Exception as e:
                    print(e)

        time.sleep(1)
            
# Сохранение файла
data.to_excel(f'{source}.xlsx')

'NoneType' object has no attribute 'find_all'


In [67]:
data.shape

(259, 5)

### india-briefing

requests

In [167]:
data = pd.DataFrame()

source = 'india-briefing'

for page in list(range(1, 30)):

    try:
        base_url = f"https://www.india-briefing.com/news/category/economy-trade/page/{page}/"
    
        r = requests.get(base_url)
        
        soup_all = BeautifulSoup(r.text, 'lxml')
    
        items = soup_all.find_all('div', class_=['content news mb-2rem'])
    
        for item in items:
    
            date = item.find("time", class_="entry-date")["datetime"]
            
            title = item.find("h3").text
            
            href = item.find("a")["href"]
    
            r = requests.get(href)
                    
            soup = BeautifulSoup(r.text, 'lxml')
            
            content =  soup.find('div', class_=['article-content post-content']).find_all(["li", "p"])
            
            text = [i.text for i in content]
            
            sub_data = pd.DataFrame([[source, title, date, href, text]], columns=['source', 'title', 'date', 'href', 'text'])
                
            data = pd.concat([data, sub_data], axis=0)
            
        time.sleep(1)

    except Exception as e:
        print(e)
    
# Сохранение файла
data.to_excel(f'{source}.xlsx')

'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'


In [168]:
print(data.shape)
display(data)

(193, 5)


,source,title,date,href,text
0,india-briefing,\nInvesting in Tamil Nadu: Manufacturing Outlo...,2024-04-18T19:14:43+08:00,https://www.india-briefing.com/news/investing-...,[Investing in Tamil Nadu: Manufacturing Outloo...
0,india-briefing,\nIndia Identifies List of 30 Critical Mineral...,2024-04-17T11:25:55+08:00,https://www.india-briefing.com/news/india-iden...,[India Identifies List of 30 Critical Minerals...
0,india-briefing,\nIndia-Peru Trade Agreement: Bilateral Negoti...,2024-04-15T20:11:10+08:00,https://www.india-briefing.com/news/india-peru...,[India-Peru Trade Agreement: Bilateral Negotia...
0,india-briefing,\nIndia-ASEAN Trade Pact: Tariff Anomalies on ...,2024-04-12T20:31:28+08:00,https://www.india-briefing.com/news/india-asea...,[India-ASEAN Trade Pact: Tariff Anomalies on G...
0,india-briefing,\nIndia’s Regulation of Solar PV Modules: Appr...,2024-04-08T21:32:53+08:00,https://www.india-briefing.com/news/india-appr...,[India’s Regulation of Solar PV Modules: Appro...
...,...,...,...,...,...
0,india-briefing,\nEconomic Survey for 2021-22 Makes Case for I...,2022-01-31T22:52:02+08:00,https://www.india-briefing.com/news/economic-s...,[Economic Survey for 2021-22 Makes Case for In...
0,india-briefing,\nIndia – Central Asia Summit: Key Outcomes an...,2022-01-28T18:48:34+08:00,https://www.india-briefing.com/news/india-cent...,[India – Central Asia Summit: Key Outcomes and...
0,india-briefing,\nFirst Advance Estimates of India’s GDP (2021...,2022-01-17T22:22:38+08:00,https://www.india-briefing.com/news/first-adva...,[First Advance Estimates of India’s GDP (2021-...
0,india-briefing,\nMergers and Acquisitions Activity in India i...,2022-01-14T22:15:06+08:00,https://www.india-briefing.com/news/mergers-ac...,[Mergers and Acquisitions Activity in India in...


### vietnam-briefing

In [351]:
data = pd.DataFrame()

source = 'vietnam-briefing'

for page in list(range(1, 20)):

    print(page)
    
    try:
        base_url = f"https://www.vietnam-briefing.com/news/category/economy-trade/page/{page}/"
    
        r = requests.get(base_url)
        
        soup_all = BeautifulSoup(r.text, 'lxml')
    
        items = soup_all.find_all('div', class_=['content news mb-2rem'])
    
        for item in items:
    
            date = item.find("time", class_="entry-date")["datetime"]
            
            title = item.find("h3").text
            
            href = item.find("a")["href"]
    
            r = requests.get(href)
                    
            soup = BeautifulSoup(r.text, 'lxml')
            
            content =  soup.find('div', class_=['article-content']).find_all(["li", "p"])
            
            text = [i.text for i in content]
            
            sub_data = pd.DataFrame([[source, title, date, href, text]], columns=['source', 'title', 'date', 'href', 'text'])
                
            data = pd.concat([data, sub_data], axis=0)
            
        time.sleep(1)

    except Exception as e:
        print(e)
    
# Сохранение файла
data.to_excel(f'{source}.xlsx')

1
2
3
4
5
6
'NoneType' object has no attribute 'find_all'
7
8
9
10
11
12
13
14
15
'NoneType' object has no attribute 'find_all'
16
17
18
19
'NoneType' object has no attribute 'find_all'


In [352]:
print(data.shape)
display(data)

(178, 5)


,source,title,date,href,text
0,vietnam-briefing,\nESG Focus Among Foreign Businesses in Vietna...,2024-04-18T14:48:00+08:00,https://www.vietnam-briefing.com/news/esg-focu...,"[ESG (environmental, social, and governance) m..."
0,vietnam-briefing,\nVietnam-Spain Bilateral Relations: Trade and...,2024-04-16T22:47:42+08:00,https://www.vietnam-briefing.com/news/vietnam-...,[Vietnam and Spain have developed relatively c...
0,vietnam-briefing,\nWhy Cambodia’s Funan Techo Canal Project is ...,2024-04-09T19:05:32+08:00,https://www.vietnam-briefing.com/news/why-camb...,[Despite the positive outcomes predicted by th...
0,vietnam-briefing,\nAn Investor’s Guide to Vietnam’s Bắc Ninh Pr...,2024-04-08T19:27:07+08:00,https://www.vietnam-briefing.com/news/an-inves...,[Bắc Ninh has recently evolved into an importa...
0,vietnam-briefing,\nVietnam’s State of the Economy: Q1 2024 Repo...,2024-04-03T20:23:42+08:00,https://www.vietnam-briefing.com/news/vietnams...,[Vietnam’s economy saw GDP y-o-y growth at 5.6...
...,...,...,...,...,...
0,vietnam-briefing,"\nVietnam’s Economy in Q1 Expands, but Supply ...",2022-04-21T18:09:41+08:00,https://www.vietnam-briefing.com/news/vietnams...,[Vietnam’s economy expanded in Q1 2022 showing...
0,vietnam-briefing,\nQ&A: The RCEP Advantage – New Trade Opportun...,2022-04-20T14:57:59+08:00,https://www.vietnam-briefing.com/news/qa-the-r...,[Officially in force since the beginning of th...
0,vietnam-briefing,\nVietnam’s Economy to Benefit Most from RCEP:...,2022-04-14T16:09:22+08:00,https://www.vietnam-briefing.com/news/vietnams...,[A recent report from the World Bank offers a ...
0,vietnam-briefing,\nVietnam’s Equitisation and Divestment: Oppor...,2022-04-04T14:45:00+08:00,https://www.vietnam-briefing.com/news/vietnams...,[Vietnam’s divestment and equitisation plans h...


### middleeastbriefing

In [353]:
data = pd.DataFrame()

source = 'middleeastbriefing'

for page in list(range(1, 20)):

    print(page)
    
    try:
        base_url = f"https://www.middleeastbriefing.com/news/category/economy-trade/page/{page}/"
    
        r = requests.get(base_url)
        
        soup_all = BeautifulSoup(r.text, 'lxml')
    
        items = soup_all.find_all('div', class_=['content news mb-2rem'])
    
        for item in items:
    
            date = item.find("time", class_="entry-date")["datetime"]
            
            title = item.find("h3").text
            
            href = item.find("a")["href"]
    
            r = requests.get(href)
                    
            soup = BeautifulSoup(r.text, 'lxml')
            
            content =  soup.find('div', class_=['article-content']).find_all(["li", "p"])
            
            text = [i.text for i in content]
            
            sub_data = pd.DataFrame([[source, title, date, href, text]], columns=['source', 'title', 'date', 'href', 'text'])
                
            data = pd.concat([data, sub_data], axis=0)
            
        time.sleep(1)

    except Exception as e:
        print(e)
    
# Сохранение файла
data.to_excel(f'{source}.xlsx')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [354]:
print(data.shape)
display(data)

(190, 5)


,source,title,date,href,text
0,middleeastbriefing,\nUnderstanding the Dubai Unified Licence: Sco...,2024-04-16T08:39:24+00:00,https://www.middleeastbriefing.com/news/dubai-...,[The Dubai Unified Licence streamlines busines...
0,middleeastbriefing,\nUnveiling Opportunities: The United Arab Emi...,2024-04-16T07:57:54+00:00,https://www.middleeastbriefing.com/news/unveil...,[An attractive tax and regulatory environment ...
0,middleeastbriefing,\nSaudi Arabia Gives Status Update on NEOM Pro...,2024-04-15T12:45:06+00:00,https://www.middleeastbriefing.com/news/saudi-...,"[NEOM, Saudi Arabia’s urban megaproject, has r..."
0,middleeastbriefing,\nNon-Oil Sector Driving Bahrain’s GDP Growth:...,2024-04-12T07:07:34+00:00,https://www.middleeastbriefing.com/news/non-oi...,[Key non-oil sectors poised to drive GDP growt...
0,middleeastbriefing,\nOman-India FTA Negotiations Reportedly Concl...,2024-04-09T09:32:59+00:00,https://www.middleeastbriefing.com/news/oman-i...,[Indian news media are reporting that Oman-Ind...
...,...,...,...,...,...
0,middleeastbriefing,"\nUAE, Egypt, Sign US$10 Billion Deal For 10GW...",2023-06-07T11:48:18+00:00,https://www.middleeastbriefing.com/news/uae-eg...,[The MENA (Middle East & North Africa) region ...
0,middleeastbriefing,\nQualifying Income Tax In The UAE Free Zone: ...,2023-06-06T12:42:49+00:00,https://www.middleeastbriefing.com/news/qualif...,[The UAE’s new Corporate Income Tax code has k...
0,middleeastbriefing,\nUAE Announces Tax Ruling On Non-Residents Pr...,2023-06-05T13:39:25+00:00,https://www.middleeastbriefing.com/news/uae-an...,[The UAE Ministry of Finance has announced a C...
0,middleeastbriefing,\nIran’s Imports From Kyrgyzstan Up 119% As IN...,2023-06-04T17:41:19+00:00,https://www.middleeastbriefing.com/news/irans-...,[Iran’s imports from Kyrgyzstan increased by 1...


### agbi

requests

In [356]:
data = pd.DataFrame()

source = 'agbi'

for page in list(range(1, 30)):

    try:
        base_url = f'https://www.agbi.com/sectors/finance/page/{page}/'
    
        r = requests.get(base_url)
    
        soup_all = BeautifulSoup(r.text, 'lxml')
    
        items = soup_all.find_all('div', class_=['card__body'])
    
        for item in items:
    
            href = item.find_all("a")[1]["href"]
            
            title = item.find_all("a")[1].text
            
            r = requests.get(href)
                    
            soup = BeautifulSoup(r.text, 'lxml')
            
            content =  soup.find('div', class_=['article__main']).find_all(["p"]) #, "li"
            
            text = [i.text for i in content]

            header = soup.find('div', class_=['article__header'])
            
            date = header.find('time', class_=['article__date']).text
    
            sub_data = pd.DataFrame([[source, title, date, href, text]], columns=['source', 'title', 'date', 'href', 'text'])
    
            data = pd.concat([data, sub_data], axis=0)
            
    except Exception as e:
        print(e)
    
# Сохранение файла
data.to_excel(f'{source}.xlsx')

list index out of range


In [357]:
print(data.shape)
display(data)

(577, 5)


,source,title,date,href,text
0,agbi,\n\t\tWhat Egypt needs to do to please its $60...,"\n\t\tMarch 24, 2024",https://www.agbi.com/analysis/economy/2024/03/...,[A fast-paced four weeks may have transformed ...
0,agbi,\n\t\tEgypt must deal with its fundamental cha...,"\n\t\tMarch 21, 2024",https://www.agbi.com/opinion/economy/2024/03/a...,[Abu Dhabi’s announcement that it would invest...
0,agbi,\n\t\tAbu Dhabi’s futuristic vision of an Egyp...,"\n\t\tMarch 22, 2024",https://www.agbi.com/analysis/tourism/2024/03/...,"[By 2052, Egypt’s Ras El Hekma will be “compet..."
0,agbi,\n\t\tCurrency bond traders flow into Egypt af...,"\n\t\tMarch 14, 2024",https://www.agbi.com/analysis/economy/2024/03/...,[JPMorgan Chase & Co has issued a circular urg...
0,agbi,\n\t\tEgypt’s $8bn rescue package brings new c...,"\n\t\tMarch 7, 2024",https://www.agbi.com/opinion/economy/2024/03/s...,[Egypt has finally secured a hefty Internation...
...,...,...,...,...,...
0,agbi,\n\t\tLondon’s suburban real estate moves Midd...,"\n\t\tNovember 28, 2023",https://www.agbi.com/real-estate/2023/11/londo...,[While central London property retains its all...
0,agbi,\n\t\tSaudi’s top 10 banks post 3.8% rise in p...,"\n\t\tNovember 27, 2023",https://www.agbi.com/banking-finance/2023/11/s...,[Saudi Arabia’s 10 largest listed banks saw ne...
0,agbi,\n\t\tAfrica should be a prime focus for Cop28...,"\n\t\tNovember 27, 2023",https://www.agbi.com/opinion/sustainability/20...,"[Climate change is, of course, a global proble..."
0,agbi,"\n\t\tMore women join UAE boards, but progress...","\n\t\tNovember 27, 2023",https://www.agbi.com/employment/2023/11/women-...,[When Maha Abdul Majeed Ismail Ali Al Fahim jo...


### mea-markets

requests

In [262]:
data = pd.DataFrame()

source = 'mea-markets'

for page in list(range(1, 30)):

    try:
        base_url = f'https://www.mea-markets.com/business-news-insight/page/{page}/'
    
        r = requests.get(base_url)
    
        soup_all = BeautifulSoup(r.text, 'lxml')
    
        items = soup_all.find_all('div', class_=['element_1656339709663'])
    
        for item in items:
    
            href = item.find_all("a")[1]["href"]
    
            title = item.find_all("a")[1].text
            
            r = requests.get(href)
                                
            soup = BeautifulSoup(r.text, 'lxml')
            
            content =  soup.find('div', class_=['article-middle-content']).find_all(["p"])
            
            text = [i.text for i in content if i.text!='']
            
            date = soup.find('span', class_=['postedat']).text
    
            sub_data = pd.DataFrame([[source, title, date, href, text]], columns=['source', 'title', 'date', 'href', 'text'])
        
            data = pd.concat([data, sub_data], axis=0)

    except Exception as e:
        print(e)
    
# Сохранение файла
data.to_excel(f'{source}.xlsx')

In [263]:
print(data.shape)
display(data)

(235, 5)


,source,title,date,href,text
0,mea-markets,Deloitte Launches Kiyadat to Advance GCC Natio...,"Posted on April 18, 2024",https://www.mea-markets.com/deloitte-launches-...,"[In Arabic, “Kiyadat” translates to “Leaders”,..."
0,mea-markets,United Nations Leaders Call for More Action to...,"Posted on April 17, 2024",https://www.mea-markets.com/united-nations-lea...,[UN Secretary-General António Guterres celebra...
0,mea-markets,Red Sea Crisis Underlines the Need for Greater...,"Posted on April 9, 2024",https://www.mea-markets.com/red-sea-crisis-und...,[The speed with which shipping companies have ...
0,mea-markets,Local SMEs Upskilled Through Enterprise Develo...,"Posted on March 26, 2024",https://www.mea-markets.com/local-smes-upskill...,"[The programme, a collaborative effort between..."
0,mea-markets,The Sky’s The Limit For Domestic and Inbound T...,"Posted on March 20, 2024",https://www.mea-markets.com/the-skys-the-limit...,"[While location and facilities are important, ..."
...,...,...,...,...,...
0,mea-markets,Falling Oil Prices: The Reaction of the GCC Co...,"Posted on September 14, 2015",https://www.mea-markets.com/2015-falling-oil-p...,[Sharp decline in oil prices reveals the impor...
0,mea-markets,Collective Bargaining Vital Tool for MENA,"Posted on September 8, 2015",https://www.mea-markets.com/2015-collective-ba...,[The ILO brought together employer and worker ...
0,mea-markets,The Spectre of Increasing Disputes?,"Posted on September 7, 2015",https://www.mea-markets.com/2015-regional-outl...,"[By Laura Warren Partner at Clyde & Co, Doha, ..."
0,mea-markets,Read more,"Posted on September 6, 2015",https://www.mea-markets.com/2015-business-conf...,[According to Global Economic Conditions Surve...


### chinadaily

requests

In [297]:
data = pd.DataFrame()

source = 'chinadaily'

for page in list(range(1, 30)):

    try:
        base_url = f'https://www.chinadaily.com.cn/business/companies/page_{page}.html'
        
        r = requests.get(base_url)
        
        soup_all = BeautifulSoup(r.text, 'lxml')
        
        items = soup_all.find_all('div', class_=['mb10 tw3_01_2'])
        
        for item in items:
        
            href = item.find_all("a")[1]["href"].replace('//', 'https://')
            
            title = item.find_all("a")[1].text
        
            r = requests.get(href)
                                
            soup = BeautifulSoup(r.text, 'lxml')
            
            content =  soup.find('div', id=['Content']).find_all(["p"])
            
            text = [i.text for i in content if i.text!='']
            
            date = soup.find('span', class_=['info_l']).text
            
            sub_data = pd.DataFrame([[source, title, date, href, text]], columns=['source', 'title', 'date', 'href', 'text'])
            
            data = pd.concat([data, sub_data], axis=0)
    
    except Exception as e:
        print(e)
    
# Сохранение файла
data.to_excel(f'{source}.xlsx')

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
list index out of range


In [298]:
print(data.shape)
display(data)

(379, 5)


,source,title,date,href,text
0,chinadaily,China starts mass production of carbon-14 isot...,\n \n Xinhua |\n Up...,https://www.chinadaily.com.cn/a/202404/21/WS66...,[BEIJING -- China has started the mass product...
0,chinadaily,Japan's Lawson to ride consumption recovery,\n By WU YONG in Shenyang and ZHENG X...,https://www.chinadaily.com.cn/a/202404/20/WS66...,[Japan's Lawson Convenience Store has plans to...
0,chinadaily,Lenovo teams up with US chip giants for AI PCs,\n By MA SI | \n CHINA DAILY...,https://www.chinadaily.com.cn/a/202404/20/WS66...,[Chinese technology company Lenovo Group Ltd i...
0,chinadaily,Weixin Pay launches service for visitors to Ca...,\n By QIU QUANLIN in Guangzhou | \n ...,https://www.chinadaily.com.cn/a/202404/20/WS66...,[Mobile payments app Weixin Pay has launched a...
0,chinadaily,Mercedes-Benz upgrades R&D center in Shanghai,\n \n Xinhua |\n Up...,https://www.chinadaily.com.cn/a/202404/20/WS66...,[SHANGHAI -- Mercedes-Benz inaugurated a new b...
...,...,...,...,...,...
0,chinadaily,Shein makes confidential filing for US IPO,\n \n chinadaily.com.cn |\n ...,https://www.chinadaily.com.cn/a/202311/29/WS65...,[Chinese fast-fashion retailer Shein has filed...
0,chinadaily,A&M shows commitment to Chinese market,\n By Cheng Yu | \n chinadai...,https://www.chinadaily.com.cn/a/202311/29/WS65...,"[Alvarez & Marsal, a global provider of profes..."
0,chinadaily,PDD Holdings shows increase in revenue and growth,\n By Fan Feifei | \n chinad...,https://www.chinadaily.com.cn/a/202311/29/WS65...,[Chinese online discounter PDD Holdings Inc sa...
0,chinadaily,State Grid Weifang Power Supply Company boosts...,\n \n chinadaily.com.cn |\n ...,https://www.chinadaily.com.cn/a/202311/29/WS65...,[With the rising utilization of new energy veh...
